In [1]:
!pip install -q faiss-cpu sentence-transformers pandas numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

df = pd.read_csv("shl_assessments_full_data.csv")

df['text_for_embedding'] = df.apply(lambda row:
    f"Assessment Name: {row['Assessment Name']}. "
    f"Description: {row['Description']}. "
    f"Type: {row['Test Type']}. "
    f"Duration: {row['Assessment Length']}. "
    f"Level: {row['Job Level']}.",
    axis=1
)


In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
import numpy as np

model = SentenceTransformer("BAAI/bge-small-en")

catalog_texts = df['text_for_embedding'].tolist()
embeddings = model.encode(catalog_texts, normalize_embeddings=True)

np.save("bge_embeddings.npy", embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings.astype('float32'))


In [5]:
def recommend_assessments(query, top_k=10):
    query_vector = model.encode([query], normalize_embeddings=True).astype('float32')
    scores, indices = index.search(query_vector, top_k)
    results = df.iloc[indices[0]].copy()

    def summarize_description(text, max_words=30):
        if pd.isna(text): return ""
        words = str(text).split()
        return ' '.join(words[:max_words]) + ("..." if len(words) > max_words else "")

    results["Short Description"] = results["Description"].apply(summarize_description)
    return results[['Assessment Name', 'Link', 'Short Description']]


In [6]:
from difflib import get_close_matches

def fuzzy_match(predicted, expected):
    for gt in expected:
        matches = get_close_matches(gt, predicted, cutoff=0.7)
        if matches:
            return True
    return False

def recall_at_k(predictions, ground_truth, k=3):
    return 1.0 if fuzzy_match(predictions[:k], ground_truth) else 0.0

def map_at_k(predictions, ground_truth, k=3):
    score = 0.0
    hit_count = 0
    for i, pred in enumerate(predictions[:k]):
        if fuzzy_match([pred], ground_truth):
            hit_count += 1
            score += hit_count / (i + 1)
    return score / min(len(ground_truth), k)


In [12]:
benchmark = [
    {
        "query": "I'm hiring a warehouse worker to assist with packaging and facility upkeep. Needs to follow safety protocols.",
        "expected": ["Workplace Safety Solution", "Industrial - Entry Level 7.1 (International)"]
    },
    {
        "query": "We need a mid-level branch manager for a bank who can lead teams and make good decisions under pressure.",
        "expected": ["Branch Manager - Short Form"]
    },
    {
        "query": "Looking for a Spanish-speaking customer service agent to support inbound travel-related calls.",
        "expected": ["Bilingual Spanish Reservation Agent Solution"]
    },
    {
        "query": "We’re hiring an assembly line worker for a manufacturing company. Must have attention to detail and safety awareness.",
        "expected": ["Industrial - Semi-skilled 7.1 (International)", "Industrial - Entry Level 7.0 Solution"]
    },
    {
        "query": "Looking for a skilled technician who can operate machinery and maintain equipment in a factory setting.",
        "expected": ["Industrial - Professional and Skilled 7.0 Solution"]
    },
    {
        "query": "Need a first-line supervisor to manage a team in a distribution center. Should be strong in leadership and problem-solving.",
        "expected": ["Supervisor - Short Form", "Claims/Operations Supervisor Solution"]
    },
    {
        "query": "Hiring an entry-level accounting assistant for bookkeeping and auditing tasks in an office setting.",
        "expected": ["Bookkeeping, Accounting, Auditing Clerk Short Form"]
    },
    {
        "query": "I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.",
        "expected": ["Technical Sales Associate Solution", "Support Associate Solution"]
    },
    {
        "query": "Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes.",
        "expected": []
    },
    {
        "query": "Here is a JD text, can you recommend some assessment that can help me screen applications. Time limit is less than 30 minutes.",
        "expected": ["Industrial - Entry Level 7.1 (International)", "Workplace Safety Solution"]
    },
    {
        "query": "I am hiring for an analyst and wants applications to screen using Cognitive and personality tests, what options are available within 45 mins.",
        "expected": ["General Cognitive Ability Test", "Analytical Thinking Assessment"]
    }
]


In [14]:
recalls = []
maps = []
skipped_cases = 0

for i, case in enumerate(benchmark):
    query = case["query"]
    expected = case["expected"]

    results = recommend_assessments(query, top_k=10)
    predicted_names = results['Assessment Name'].tolist()

    print(f"\n🔹 Test Case {i + 1}")
    print("Query:", query)
    print("Expected:", expected)
    print("Top 10 Predicted:", predicted_names)

    if not expected:
        print("⚠️  No expected results provided for this query — skipping from accuracy scoring.")
        skipped_cases += 1
        continue

    r_at_3 = recall_at_k(predicted_names, expected, k=3)
    m_at_3 = map_at_k(predicted_names, expected, k=3)

    print(f"Recall@3: {r_at_3:.2f} | MAP@3: {m_at_3:.2f}")

    recalls.append(r_at_3)
    maps.append(m_at_3)

total_cases = len(benchmark) - skipped_cases

print("\n====== Final Scores ======")
print(f"Evaluated on {total_cases} valid test cases (skipped {skipped_cases})")
print(f"✅ Mean Recall@3: {np.mean(recalls):.2f}")
print(f"✅ Mean MAP@3: {np.mean(maps):.2f}")



🔹 Test Case 1
Query: I'm hiring a warehouse worker to assist with packaging and facility upkeep. Needs to follow safety protocols.
Expected: ['Workplace Safety Solution', 'Industrial - Entry Level 7.1 (International)']
Top 10 Predicted: ['Workplace Safety Solution', 'Workplace Safety Solution', 'Workplace Safety - Individual 7.0 Solution', 'Workplace Safety - Individual 7.0 Solution', 'Industrial - Entry Level 7.1 (International)', 'Industrial - Entry Level 7.1 (International)', 'Industrial - Entry Level 7.1 (International)', 'Industrial - Entry Level 7.1 (International)', 'Industrial - Entry Level 7.1 (International)', 'Industrial - Entry Level 7.1 (International)']
Recall@3: 1.00 | MAP@3: 1.50

🔹 Test Case 2
Query: We need a mid-level branch manager for a bank who can lead teams and make good decisions under pressure.
Expected: ['Branch Manager - Short Form']
Top 10 Predicted: ['Branch Manager - Short Form', 'Branch Manager - Short Form', 'Branch Manager - Short Form', 'Branch Manag